In [27]:
import matplotlib
# Force matplotlib to not use any Xwindows backend.
matplotlib.use('Agg')
import numpy as np
import wradlib as wrl
import osgeo.gdal as gdal
from prov.dot import prov_to_dot

import shutil
import glob
import os
import datetime
import multiprocessing
from PIL import Image
from provstore.api import Api

import prov.model as prov

d1 = prov.ProvDocument()
d1.set_default_namespace('http://prov.org/')
d1.add_namespace('array2raster','http://array2raster.org/')
d1.add_namespace('hdf2', 'http://hdf2filepath.org/')
d1.add_namespace('attributes','http://attributes.org/')

def array_to_raster(array,grid,raster_file,d2,filename,e_gridded,e_gridxy):
    """Array > Raster
    Save a raster from a C order array.

    :param array: ndarray
    """
    e_array=d2.entity('array2raster:e_array',(('filepath',filename),('attributes',"attributes:array")))
    e_grid=d2.entity('array2raster:e_array',(('filepath',filename),('attributes',"attributes:array")))
    a_array_shape=d1.activity('hdf2:a_array_shape')
    d2.used(a_array_shape,e_array)
    d2.wasDerivedFrom(e_array,e_gridded)
    d2.wasDerivedFrom(e_grid,e_gridxy)
    
    x_pixels,y_pixels = array.shape
    
    e_xpixel=d2.entity('array2raster:e_xpixel',(('filepath',filename),('attributes',"attributes:xpixel")))
    e_ypixel=d2.entity('array2raster:e_ypixel',(('filepath',filename),('attributes',"attributes:ypixel")))
    d2.wasGeneratedBy(e_xpixel,a_array_shape)
    d2.wasGeneratedBy(e_ypixel,a_array_shape)
    
    e_size=d2.entity('array2raster:e_size',(('filepath',filename),('attributes',"attributes:size")))
    e_grid=d2.entity('array2raster:e_grid',(('filepath',filename),('attributes',"attributes:grid")))
    
    PIXEL_SIZE = grid[1,0] - grid[0,0]
    
    d1.wasDerivedFrom(e_size,e_grid)

    print(PIXEL_SIZE)
    
    e_xmin=d2.entity('array2raster:e_xmin',(('filepath',filename),('attributes',"attributes:xmin")))
    e_ymax=d2.entity('array2raster:e_ymax',(('filepath',filename),('attributes',"attributes:ymax")))
    
    x_min = grid[0,0]
    y_max = grid[0,1]  # x_min & y_max are like the "top left" corner.
    
    d2.wasDerivedFrom(e_xmin,e_grid)
    d2.wasDerivedFrom(e_ymax,e_grid)

    wkt_projection = 'PROJCS["OSGB 1936 / British National Grid",GEOGCS["OSGB 1936",DATUM["OSGB_1936",SPHEROID["Airy 1830",6377563.396,299.3249646,AUTHORITY["EPSG","7001"]],AUTHORITY["EPSG","6277"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.01745329251994328,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4277"]],UNIT["metre",1,AUTHORITY["EPSG","9001"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",49],PARAMETER["central_meridian",-2],PARAMETER["scale_factor",0.9996012717],PARAMETER["false_easting",400000],PARAMETER["false_northing",-100000],AUTHORITY["EPSG","27700"],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'
    
    driver = gdal.GetDriverByName('GTiff')
    
    a_driver_create=d2.activity('array2raster:a_driver_create')
    e_dataset=d2.entity('array2raster:e_dataset',(('filepath',filename),('attributes',"attributes:dataset")))
    d2.used(a_driver_create,e_xpixel)
    d2.used(a_driver_create,e_ypixel)
    d2.wasGeneratedBy(e_dataset,a_driver_create)

    dataset = driver.Create(
        raster_file,
        x_pixels,
        y_pixels,
        1,
        gdal.GDT_Float32, )
    
    
    dataset.SetGeoTransform((
        x_min,    # 0
        PIXEL_SIZE,  # 1
        0,                      # 2
        y_max,    # 3
        0,                      # 4
        PIXEL_SIZE))
    
    a_geotransform=d2.activity('array2raster:a_geotransform')

    
    d2.used(a_geotransform,e_xmin)
    d2.used(a_geotransform,e_ymax)
    d2.used(a_geotransform,e_size)
    d2.wasGeneratedBy(e_dataset,a_geotransform)
    
    dataset.SetProjection(wkt_projection)
    dataset.GetRasterBand(1).WriteArray(array)
    dataset.FlushCache()  # Write to disk.
    return dataset, dataset.GetRasterBand(1),raster_file,d1,filename #If you need to return, remenber to return  also the dataset because the band don`t live without dataset.

def hdf2array(filename,length):
    
    e_filename=d1.entity('hdf2:e_filename',(('filepath',filename),('attributes',"attributes:filename")))
    e_data=d1.entity('hdf2:e_data',(('filepath',filename),('attributes',"attributes:data")))
    a_read=d1.activity('hdf2:a_read')
    
    data = wrl.io.read_OPERA_hdf5(filename)
    
    d1.used(a_read,e_filename)
    
    d1.wasGeneratedBy(e_data,a_read)
    
    d1.entity("hdf2:e_data")
    d1.activity("hdf2:a_read")
    d1.entity("hdf2:filename")
    d1.used("hdf2:a_read","hdf2:filename")
    d1.used("hdf2:e_data","hdf2:a_read")
    
    numpy_data =  np.array(data['dataset1/data1/data'][:])
    e_numpy_data = d1.entity('hdf2:e_numpy_data')
    a_np = d1.activity('hdf2:e_np')
    d1.wasGeneratedBy(e_numpy_data,a_np)
    e_dbz=d1.entity('hdf2:e_dbz',(('filepath',filename),('attributes',"attributes:dbz")))
    
    d1.activity("hdf2:a_np")
    d1.entity("hdf2:e_numpy_data")
    d1.used("hdf2:a_np","hdf2:e_data")
    d1.wasGeneratedBy("hdf2:e_numpy_data","hdf2:a_np")
    
    dbZ = (numpy_data * data['dataset1/data1/what']['gain']) +data['dataset1/data1/what']['offset']
    dbZ[dbZ < 5] = 0

    Z = wrl.trafo.idecibel(dbZ)

    e_z=d1.entity('hdf2:e_z',(('filepath',filename),('attributes',"attributes:z")))
    a_wrl_trafo=d1.activity('hdf2:a_wrl_trafo')
    d1.wasGeneratedBy(e_z,a_wrl_trafo)
    d1.used(a_wrl_trafo,e_dbz)
    
    R = wrl.zr.z2r(Z, a=200., b=1.6)
    
    e_r=d1.entity('hdf2:e_r',(('filepath',filename),('attributes',"attribute:r")))

    a_wrl_zr=d1.activity('hdf2:a_wrl_zr')
    d1.used(a_wrl_zr,e_z)
    d1.wasGeneratedBy(e_r,a_wrl_zr)
    
    depth = wrl.trafo.r2depth(R, 600)
    
    e_depth=d1.entity('hdf2:e_depth',(('filepath',filename),('attributes',"attributes:depth")))
    
    
    d1.used(a_wrl_trafo,e_r)
    d1.wasGeneratedBy(e_depth,a_wrl_trafo)

    depth[dbZ==0]=0
    
    d1.wasDerivedFrom(e_depth,e_dbz)

    where = data['where']
    
    e_where=d1.entity('e_where',(('filepath',filename),('attributes',"attributes:where")))
    d1.wasDerivedFrom(e_where,e_data)

    
    radar_location = (where['lon'], where['lat'],where['height'])
    
    e_location=d1.entity('hdf2:e_location',(('filepath',filename),('attributes',"attributes:location")))
    d1.wasDerivedFrom(e_location,e_where)

    
    elevation = data['dataset1/where']['elangle']
    
    e_elevation=d1.entity('hdf2:e_elevation',(('filepath',filename),('attributes',"attributes:elevation")))
    d1.wasDerivedFrom(e_elevation,e_data)
    
    azimuths = np.arange(0,360)

    e_azimuths=d1.entity('hdf2:e_azimuths',(('filepath',filename),('attributes',"attributes:azimuths")))
    d1.wasGeneratedBy(e_azimuths,a_np)
    
    ranges = np.arange(0, length, data['dataset1/where']['rscale']) # in meters
    
    e_ranges=d1.entity('hdf2:e_ranges',(('filepath',filename),('attributes',"attributes:ranges")))
    d1.used(a_np,e_data)
    d1.wasGeneratedBy(e_ranges,a_np)
    
    polargrid = np.meshgrid(ranges, azimuths)

    e_polargrid=d1.entity('hdf2:e_polargrid',(('filepath',filename),('attributes',"attributes:polargrid")))
    d1.used(a_np,e_azimuths)
    d1.used(a_np,e_ranges)
    d1.wasGeneratedBy(e_polargrid,a_np)
    
    lon, lat, alt = wrl.georef.polar2lonlatalt_n(polargrid[0], polargrid[1],
                                                 elevation, radar_location)
    
    e_lon=d1.entity('hdf2:e_lon',(('filepath',filename),('attributes',"attributes:lon")))
    e_lat=d1.entity('hdf2:e_lat',(('filepath',filename),('attributes',"attributes:lat")))
    e_alt=d1.entity('hdf2:e_alt',(('filepath',filename),('attributes',"attributes:alt")))
    a_wrl_georef=d1.activity('hdf2:wrl_georef')
    
    d1.used(a_wrl_georef,e_polargrid)
    d1.used(a_wrl_georef,e_elevation)
    d1.used(a_wrl_georef,e_location)
    d1.wasGeneratedBy(e_lon,a_wrl_georef)
    d1.wasGeneratedBy(e_lat,a_wrl_georef)
    d1.wasGeneratedBy(e_alt,a_wrl_georef)
    
    bng = wrl.georef.epsg_to_osr(27700)
    
    e_bng=d1.entity('hdf2:e_bng',(('filepath',filename),('attributes',"attributes:bng")))
    d1.wasGeneratedBy(e_bng,a_wrl_georef)
    
    x, y = wrl.georef.reproject(lon, lat, projection_target=bng)
    
    e_x=d1.entity('hdf2:e_x',(('filepath',filename),('attributes',"attributes:x")))
    e_y=d1.entity('hdf2:e_y',(('filepath',filename),('attributes',"attributes:y")))
    d1.used(a_wrl_georef,e_lon)
    d1.used(a_wrl_georef,e_lat)
    d1.used(a_wrl_georef,e_bng)
    d1.wasGeneratedBy(e_x,a_wrl_georef)
    d1.wasGeneratedBy(e_y,a_wrl_georef)
    
    xgrid = np.linspace(x.min(), x.max(), 500)
    ygrid = np.linspace(y.min(), y.max(), 500)
    
    e_xgrid=d1.entity('hdf2:e_xgrid',(('filepath',filename),('attributes',"attributes:xgrid")))
    e_ygrid=d1.entity('hdf2:e_ygrid',(('filepath',filename),('attributes',"attributes:ygrid")))
    d1.used(a_np,e_x)
    d1.used(a_np,e_y)
    d1.wasGeneratedBy(e_xgrid,a_np)
    d1.wasGeneratedBy(e_ygrid,a_np)
    
    grid_xy = np.meshgrid(xgrid, ygrid)
    grid_xy = np.vstack((grid_xy[0].ravel(), grid_xy[1].ravel())).transpose()
    
    e_gridxy=d1.entity('hdf2:e_gridxy',(('filepath',filename),('attributes',"attributes:gridxy")))
    d1.used(a_np,e_xgrid)
    d1.used(a_np,e_ygrid)
    d1.wasGeneratedBy(e_gridxy,a_np)
    
    xy=np.concatenate([x.ravel()[:,None],y.ravel()[:,None]], axis=1)
    
    e_xy=d1.entity('hdf2:e_xy',(('filepath',filename),('attributes',"attributes:xy")))
    d1.used(a_np,e_x)
    d1.used(a_np,e_y)
    d1.wasGeneratedBy(e_xy,a_np)
    e_gridded=d1.entity('hdf2:e_gridded',(('filepath',filename),('attributes',"attributes:gridded")))
    
    gridded = wrl.comp.togrid(xy, grid_xy, length, np.array([x.mean(), y.mean()]), depth.ravel(), wrl.ipol.Nearest)
    
    a_wrl_togrid=d1.activity('hdf2:wrl_togrid')
    d1.used(a_wrl_togrid,e_xy)
    d1.used(a_wrl_togrid,e_gridxy)
    
    d1.used(a_wrl_togrid,e_x)
    d1.used(a_wrl_togrid,e_y)
    d1.used(a_wrl_togrid,e_depth)
    d1.wasGeneratedBy(e_gridded,a_wrl_togrid)
    
    gridded = np.ma.masked_invalid(gridded).reshape((len(xgrid), len(ygrid)))
    
    d1.used(a_np,e_xgrid)
    d1.used(a_np,e_ygrid)
    d1.wasGeneratedBy(e_gridded,a_np)
    
    gridded[gridded.mask] = -999
    return gridded,grid_xy,d1,e_gridded,e_gridxy

def copy_over_files():
    for hdf_file in  glob.glob('/srv/radar/shared_folder/HDF*C.z'):
        file_name = hdf_file.split('/')[-1]
        new_file = os.path.join('/Users/wangzhen/Desktop/docker-gdal/data4/radar_files',file_name)

        if not os.path.exists(new_file):

            shutil.copyfile(hdf_file,new_file)
            print(hdf_file)


def make_tiff(file_path,raster_path,length):
    #store the image
#     import os
    
    # visualize the graph
#     from prov.dot import prov_to_dot
    
    
    print('making',file_path)
    gridded, grid_xy,d1,e_gridded,e_gridxy= hdf2array(file_path, length)
    dataset, band,  raster_path,d2,filename = array_to_raster(gridded,grid_xy,raster_path,d1,file_path,e_gridded,e_gridxy)

    os.environ["PATH"] += os.pathsep + '/usr/local/Cellar/graphviz/2.40.1/bin'
    
    
    
   
    #prov images
    dot = prov_to_dot(d2)
    png = str(counter)+'.png'
    dot.write_png(png)
    from IPython.display import Image
    Image('prov.png')
    counter = counter + 1
        
    api = Api(username="ryan", api_key="70dd9d23b545db0f4baca887f229641f374a3dca")
    provfile = 'provdocument' + str(counter)
    stored_document = api.document.create(d2, name=provfile,public = True)
    id='prov_id is: '+str(stored_document.id)
    print(id)

def main_script():
    global counter
    counter = 0
    print('Processing radar...')
    files = os.listdir('/Users/wangzhen/Desktop/docker-gdal/data4/radar_tiffs')

    for file in files:
        try:
            Image.open('/Users/wangzhen/Desktop/docker-gdal/data4/radar_tiffs' + file)
        except:
            print('deleting')
            os.remove('/Users/wangzhen/Desktop/docker-gdal/data4/radar_tiffs' + file)

    jobs = []

    for file in os.listdir('/Users/wangzhen/Desktop/docker-gdal/data4/radar_files'):
        print(file)
        file_path = os.path.join('/Users/wangzhen/Desktop/docker-gdal/data4/radar_files', file)
        b1, b2, b3, dt_string, b4, length, dist, b5, angle, b6 = file.split('-')
        length = float(length) * 100
        raster_file = '%s_%s_%s.tiff' % (dt_string, int(length), angle)

        raster_path = os.path.join('/Users/wangzhen/Desktop/docker-gdal/data4/radar_tiffs', raster_file)
        if not os.path.exists(raster_path):
            p = multiprocessing.Process(target=make_tiff, args=(file_path,raster_path,length,))
            jobs.append(p)
            p.start()

main_script()

Processing radar...
HDF-PPI-A00-201607200005-B-1080-0450-0010-0015-C.z
HDF-PPI-A00-201607200005-B-1080-0450-0010-0015-U.z
HDF-PPI-A00-201607200005-B-1080-0450-0010-0040-C.z
making /Users/wangzhen/Desktop/docker-gdal/data4/radar_files/HDF-PPI-A00-201607200005-B-1080-0450-0010-0015-C.z
HDF-PPI-A00-201607200005-B-1080-0450-0010-0030-U.z
making /Users/wangzhen/Desktop/docker-gdal/data4/radar_files/HDF-PPI-A00-201607200005-B-1080-0450-0010-0015-U.z
HDF-PPI-A00-201607200005-B-1080-0450-0010-0020-C.z
making /Users/wangzhen/Desktop/docker-gdal/data4/radar_files/HDF-PPI-A00-201607200005-B-1080-0450-0010-0040-C.z
HDF-PPI-A00-201607200005-B-1080-0450-0010-0030-C.z
HDF-PPI-A00-201607200005-B-1080-0450-0010-0020-U.z
making /Users/wangzhen/Desktop/docker-gdal/data4/radar_files/HDF-PPI-A00-201607200005-B-1080-0450-0010-0030-U.z
making /Users/wangzhen/Desktop/docker-gdal/data4/radar_files/HDF-PPI-A00-201607200005-B-1080-0450-0010-0020-C.z
making /Users/wangzhen/Desktop/docker-gdal/data4/radar_files/HD